In [1]:
import tensorflow as tf
import numpy as np

In [2]:
def conv_relu(input, kernel_shape, bias_shape):
    # created variable named "weight"
    weights = tf.get_variable("weights",kernel_shape,
                             initializer = tf.random_normal_initializer())
    biases = tf.get_variable("biases",bias_shape, initializer = tf.constant_initializer(0.0))
    conv = tf.nn.conv2d(input, weights, strides = [1,1,1,1], padding = "SAME")
    return tf.nn.relu(conv+biases)

In [3]:
def my_image_filter(input_images, reuse = True):
    with tf.variable_scope("conv1", reuse = reuse):
        # Variables created here will be named "conv1/weights"
        relu1 = conv_relu(input_images, [5,5,32,32], [32])
    with tf.variable_scope("conv2", reuse = reuse):
        #Variables created here will be named "conv2/weights"
        return conv_relu(relu1, [5,5,32,32], [32])

In [4]:
image1 = np.random.randn(28,28,32).astype("float32")
image1= image1[None]
image2 = np.random.randn(28,28,32).astype("float32")
image2= image2[None]
print(image1.shape, image2.shape)

(1, 28, 28, 32) (1, 28, 28, 32)


In [7]:
result1 = my_image_filter(image1, reuse = True)
result2 = my_image_filter(image2, reuse = True)

In [9]:
with tf.variable_scope("foo") as foo_scope:
    v = tf.get_variable("v", [1])
with tf.variable_scope(foo_scope):
    w = tf.get_variable("w", [1])
with tf.variable_scope(foo_scope, reuse=True):
    v1 = tf.get_variable("v", [1])
    w1 = tf.get_variable("w", [1])
assert v1 is v
assert w1 is w

ValueError: Variable foo/v already exists, disallowed. Did you mean to set reuse=True in VarScope? Originally defined at:

  File "<ipython-input-8-82c6eccf7fb4>", line 2, in <module>
    v = tf.get_variable("v", [1])
  File "/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py", line 3066, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py", line 3006, in run_ast_nodes
    if self.run_code(code, result):


In [3]:
def style_transfer(content_image, style_image, image_size, style_size, content_layer, content_weight,
                   style_layers, style_weights, tv_weight, init_random = False):
    
    feats = model.extract_features(model.image)
    
    content_img = preprocess_image(load_image(content_image, size = image_size))
    content_target =  sess.run(feats[content_layer],{model.image: content_img[None]})
    
    style_img = preprocess_image(load_image(style_image, size= style_size))
    style_targets_vars = [gram_matrix(feats[idx]) for idx in style_layers]
    style_targets =  sess.run(style_targets_vars, {model.image: style_image[None]})
    
    if init_random:
        img_var = tf.Variable(tf.random_uniform(content_img[None].shape, 0,1),name="image")
    else:
        img_var = tf.Variable(content_img[None], name= "image")
        
    feats = model.extract_features(img_var)
    
    c_loss = content_loss(content_weight, feats[content_layer], content_target)
    s_loss = style_loss(feats, style_layers, style_targets, style_weights)
    t_loss = tv_loss(img_var, tv_weight)
    loss = c_loss + s_loss + t_loss
    
    initial_lr = 3.0
    decayed_lr = 0.1
    decayed_lr_at = 180
    max_iter = 200
    
    lr_var = tf.Variable(initial_lr, name="lr")
    with tf.variable_scope("optimizer") as opt_scope:
        train_op = tf.train.AdamOptimizer(lr_var).minimize(loss, var_list=[img_var])
    opt_vars = tf.8get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope= opt_scope.name)
    sess.run(tf.variables_initializer([lr_var, img_var] + opt_vars))
    
    clamp_image_op = tf.assign(imag_var, tf.clip_by_value(img_var, -1.5, 1.5))
    
    f, axarr = plt.subplots(1,2)
    axarr[0].axis('off')
    axarr[1].axis('off')
    axarr[0].set_titile('Content Source Img')
    axarr[1].set_title('Style Source Img')
    axarr[0].imshow(deprocess_image(content_img))
    axarr[1].imshow(deprpcess_image(style_img))
    plt.show()
    plt.figure()
    
    for t in range(max_iter):
        sess.run(train_op)
        if t< decay_lr_at:
            sess.run(clamp_image_op)
        if t == decay_lr_at:
            sess.run(tf.assign(lr_var, decayed_lr))
        if  t%100 == 0:
            print('Iteration {}'.format(t))
            img = sess.run(img_var)
            plt.imgshow(deprocess_img(img[0], rescale = True))
            plt.axis('off')
            plt.show()
            
    print('Iteration {}'.format(t))
    img = sess.run(img_var)
    plt.imshow(deprocess_image(img[0], rescale= True))
    plt.axis('off')
    plt.show()
        
    
    
    
    
    
    